# LangChain Agents Intro

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Agents, adding the ability to use tools such as search and calculators to complete tasks that normal LLMs cannot fufil.

In [39]:
import os
from getpass import getpass
from dotenv import load_dotenv

# Load variables from .env into environment
load_dotenv()

os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = "https://eu.api.smith.langchain.com "
os.environ['LANGSMITH_API_KEY'] =  os.getenv('LANGSMITH_API_KEY') or getpass('Enter your LangSmith API Key: ')
os.environ['LANGSMITH_PROJECT'] = 'agents-intro'

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass(
    "Enter GOOGLE API Key: "
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",      
    temperature=0.0)

## What is the Agent Executor?

When we talk about agents, a significant part of an "agent" is simple code logic,
iteratively rerunning LLM calls and processing their output. The exact logic varies
significantly, but one well-known example is the **ReAct** agent.

![ReAct process](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-00.png&w=640&q=75)

**Re**ason + **Act**ion (ReAct) agents use iterative _reasoning_ and _action_ steps to
incorporate chain-of-thought and tool-use into their execution. During the _reasoning_
step, the LLM generates the steps to take to answer the query. Next, the LLM generates
the _action_ input, which our code logic parses into a tool call. (observation -> tool output)

![Agentic graph of ReAct](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-01.png&w=640&q=75)

Following our action step, we get an observation from the tool call. Then, we feed the
observation back into the agent executor logic for a final answer or further reasoning
and action steps.

The agent and agent executor we will be building will follow this pattern.

## Introduction to Tools

Tools are a way to augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the `@tool` decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [80]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'y' from 'x'."""
    return x - y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y    

@tool
def divide(x: float, y: float) -> float:
    """Divide 'x' by 'y'."""
    if y == 0:
        raise ValueError("Cannot divide by zero.")
    return x / y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [8]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x0000017222477240>)

We can see the tool name, description, and arg schema:

In [11]:
print(f"{add.name}\n{add.description} \n")

add
Add 'x' and 'y'. 



The `args_schema` is a pydantic model that is transformed into the JSON schema and passed into our LLM, it is this that defines _how_ the tool is used for the LLM.

In [15]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [16]:
add.args

{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [25]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [38]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent.
The agent will be constructed using syntax and components like so:

```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent writes its notes as it works through multiple internal thought and tool-use steps to produce a final output for the user. This scratchpad is a list of messages with alternating roles of `ai` (for the tool call) and `tool` (for the tool execution output). Both message types require a `tool_call_id` field which is used to link the respective AI and tool messages - this can be required when we many tool calls happening in parallel.

In [83]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."),
    MessagesPlaceholder(variable_name="chat_history"), 
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"), # MessagesPlaceholder can used like this as well
])

In [84]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You're a helpful assistant. When answering a user's question you should first use one of the tools provided. After using a tool the tool output will be provided in the 'scratchpad' below. If you have an answer in the scratchpad you should not use any more tools and instead answer directly to the user."), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad', optional=True)]

To add tools to our LLM, we will use the `bind_tools` method within the LCEL constructor, which will take our tools and add them to the LLM. We'll also include the `tool_choice="any"` argument to `bind_tools`, which tells the LLM that it _MUST_ use a tool, ie it cannot provide a final answer directly (in therefore not using a tool):

In [88]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, divide, exponentiate]

#define the agent runnable
agent: RunnableSerializable = (
    {
    "input": lambda x: x["input"],
    "chat_history": lambda x: x["chat_history"],
    "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    } 
    | prompt
    | llm.bind_tools(tools, tool_choice="any")
)

We invoke the agent with the `invoke` method, passing in the input and chat history.

In [89]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"y": 10, "x": 10}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--54649d19-ae82-4f03-9a1f-fb4bcb0034b9-0', tool_calls=[{'name': 'add', 'args': {'y': 10, 'x': 10}, 'id': 'c4bdc698-193f-46b4-b5ed-1c40b773e6c2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 325, 'output_tokens': 81, 'total_tokens': 406, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 61}})

Because we set `tool_choice="any"` to force the tool output, the usual `content` field will be empty as that field is used for natural language output, ie the _final answer_ of the LLM. To find our tool output, we need to look at the `tool_calls` field:

In [90]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'y': 10, 'x': 10},
  'id': 'c4bdc698-193f-46b4-b5ed-1c40b773e6c2',
  'type': 'tool_call'}]

From here, we have the tool `name` that our LLM wants to use and the `args` that it
wants to pass to that tool. We can see that the tool `add` is being used with the
arguments `x=10` and `y=10`. The `agent.invoke` method has _not_ executed the tool
function; we need to write that part of the agent code ourselves.

Executing the tool code requires two steps:

1. Map the tool `name` to the tool function.

2. Execute the tool function with the generated `args`.

In [94]:
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}
name2tool

{'add': <function __main__.add(x: float, y: float) -> float>,
 'subtract': <function __main__.subtract(x: float, y: float) -> float>,
 'multiply': <function __main__.multiply(x: float, y: float) -> float>,
 'divide': <function __main__.divide(x: float, y: float) -> float>,
 'exponentiate': <function __main__.exponentiate(x: float, y: float) -> float>}

Now execute to get our answer:

In [101]:
tool_execution_output = name2tool[tool_call.tool_calls[0]["name"]](**tool_call.tool_calls[0]["args"])
tool_execution_output

20

That is our answer and tool execution logic. We feed this back into our LLM via the
`agent_scratchpad` placeholder.

In [ ]:
from langchain_core.messages import ToolMessage

tool_execution = ToolMessage(
    content = tool_execution_output,
    tool_call_id = tool_call.tool_calls[0]["id"]
)


In [104]:

result = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_execution]
})

In [105]:
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"y": 10, "x": 10}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--dc12b217-262b-495b-a11a-ddcbec16225f-0', tool_calls=[{'name': 'add', 'args': {'y': 10, 'x': 10}, 'id': '2296d988-0ea5-4e91-8d64-446c53e9bd3b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 359, 'output_tokens': 105, 'total_tokens': 464, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 85}})

Despite having the answer in our `agent_scratchpad`, the LLM still tries to use the tool
_again_. This behaviour happens because we bonded the tools to the LLM with
`tool_choice="any"`. When we set `tool_choice` to `"any"` or `"required"`, we tell the
LLM that it _MUST_ use a tool, i.e., it cannot provide a final answer.

There's two options to fix this:

1. Set `tool_choice="auto"` to tell the LLM that it can choose to use a tool or provide
a final answer.

2. Create a `final_answer`

First, let's try option **1**:

In [183]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

We'll start from the start again, so `agent_scratchpad` is empty:

In [184]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={'function_call': {'name': 'add', 'arguments': '{"y": 10, "x": 10}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--2f328403-dcae-4035-9738-45c3e18cfe6e-0', tool_calls=[{'name': 'add', 'args': {'y': 10, 'x': 10}, 'id': '7c8a2221-b107-4686-b8c2-a366466e9975', 'type': 'tool_call'}], usage_metadata={'input_tokens': 429, 'output_tokens': 74, 'total_tokens': 503, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 54}})

Now we execute the tool and pass it's output into the `agent_scratchpad` placeholder:


In [185]:
tool_execution_output = name2tool[tool_call.tool_calls[0]["name"]](**tool_call.tool_calls[0]["args"])
tool_execution_output

20

In [186]:
tool_execution = ToolMessage(
    content = tool_execution_output,
    tool_call_id = tool_call.tool_calls[0]["id"]
)
tool_execution

ToolMessage(content='20', tool_call_id='7c8a2221-b107-4686-b8c2-a366466e9975')

In [187]:
result = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_execution]
})
result

AIMessage(content='The answer is 20.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--a2988ddf-1e68-4ffd-b042-d71b7a171f52-0', usage_metadata={'input_tokens': 463, 'output_tokens': 7, 'total_tokens': 470, 'input_token_details': {'cache_read': 0}})

We now have the final answer in the `content` field! This method is perfectly
functional; however, we recommend option **2** as it provides more control over the
agent's output.

There are several reasons that option **2** can provide more control, those are:

* It removes the possibility of an agent using the direct `content` field when it is not
appropriate; for example, some LLMs (particularly smaller ones) may try to use the
`content` field when using a tool.

* We can enforce a specific structured output in our answers. Structured outputs are
handy when we require particular fields for downstream code or multi-part answers. For
example, a RAG agent may return a natural language answer and a list of sources used to
generate that answer.

To implement option **2**, we must create a `final_answer` tool. We will add a
`tools_used` field to give our output some structure—in a real-world use case, we
probably wouldn't want to generate this field, but it's useful for our example here.

In [188]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`."""

    return {"answer": answer, "tools_used": tools_used}

Our `final_answer` tool _doesn't_ necessarily need to do anything; in this example,
we're using it purely to structure our final response. We can now add this tool to our
agent:

In [189]:
tools = [add, subtract, multiply, divide, exponentiate, final_answer]

name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any") # we're forcing tool use again
)

In [190]:
tool_call = agent.invoke({"input": "What is 10 + 10?", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'y': 10, 'x': 10},
  'id': '1690580c-c4a0-4ec3-885a-0e45ef01aa98',
  'type': 'tool_call'}]

We execute the tool and provide it's output to the agent again:

In [191]:
tool_execution_output = name2tool[tool_call.tool_calls[0]["name"]](**tool_call.tool_calls[0]["args"])
tool_execution_output

20

In [192]:
tool_execution = ToolMessage(
    content = tool_execution_output,
    tool_call_id = tool_call.tool_calls[0]["id"]
)

result = agent.invoke({
    "input": "What is 10 + 10?",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_execution]
})
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'final_answer', 'arguments': '{"tools_used": ["add"], "answer": "10 + 10 = 20"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--554574b1-ed89-4bd1-9b3e-b2a04944b169-0', tool_calls=[{'name': 'final_answer', 'args': {'tools_used': ['add'], 'answer': '10 + 10 = 20'}, 'id': '73fb6a1f-1f24-4d43-bbac-35954f831060', 'type': 'tool_call'}], usage_metadata={'input_tokens': 464, 'output_tokens': 81, 'total_tokens': 545, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 50}})

We see that `content` remains empty because we force tool use. But we now have the
`final_answer` tool, which the agent executor passes via the `tool_calls` field:

In [193]:
result.tool_calls

[{'name': 'final_answer',
  'args': {'tools_used': ['add'], 'answer': '10 + 10 = 20'},
  'id': '73fb6a1f-1f24-4d43-bbac-35954f831060',
  'type': 'tool_call'}]

In [194]:
result.tool_calls[0]["args"]

{'tools_used': ['add'], 'answer': '10 + 10 = 20'}

In [195]:
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'final_answer', 'arguments': '{"tools_used": ["add"], "answer": "10 + 10 = 20"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--554574b1-ed89-4bd1-9b3e-b2a04944b169-0', tool_calls=[{'name': 'final_answer', 'args': {'tools_used': ['add'], 'answer': '10 + 10 = 20'}, 'id': '73fb6a1f-1f24-4d43-bbac-35954f831060', 'type': 'tool_call'}], usage_metadata={'input_tokens': 464, 'output_tokens': 81, 'total_tokens': 545, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 50}})

### Building a Custom Agent Execution Loop

We've worked through each step of our agent code, but it doesn't run without us running
every step. We must write a class to handle all the logic we just worked through.

In [212]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")
        )

    def invoke(self, input: str) -> dict:
        #invoke the agent but we do this iteratiively in a loop until reaching a final answer
        count = 0
        agent_scratchpad = []

        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # add initial too call to scratchpad 
            agent_scratchpad.append(tool_call)

            # we execute the tool and add the output to the scratchpad
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]

            tool_execution_output = name2tool[tool_name](**tool_args)
            tool_execution = ToolMessage(
                content = f"{tool_execution_output}",
                tool_call_id = tool_call_id
            )

            agent_scratchpad.append(tool_execution)

            #add a print statement to see the intermediate steps
            print(f"{count}: {tool_name}({tool_args}) => {tool_execution_output}")
            count += 1

            # check if the tool called was final_answer
            if tool_name == "final_answer":
                break

        # add the final output to the chat history
        final_output = tool_execution_output['answer']
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_output)
        ])
        
        # return the final answer in dict form
        return json.dumps(tool_execution_output)
    

Now initialize the agent executor:

In [216]:
agent_executor = CustomAgentExecutor(max_iterations=5)

And test the `invoke` method:

In [214]:
agent_executor.invoke(input="What is 10 + 10")

0: add({'y': 10, 'x': 10}) => 20
1: final_answer({'tools_used': ['add'], 'answer': '10 + 10 = 20'}) => {'answer': '10 + 10 = 20', 'tools_used': ['add']}


'{"answer": "10 + 10 = 20", "tools_used": ["add"]}'

In [217]:
agent_executor.invoke(input="What is 10 + 100 / (2 * 5)**2 ")

0: multiply({'y': 5, 'x': 2}) => 10
1: exponentiate({'y': 2, 'x': 10}) => 100
2: divide({'y': 100, 'x': 100}) => 1.0
3: add({'y': 1, 'x': 10}) => 11
4: final_answer({'tools_used': ['multiply', 'exponentiate', 'divide', 'add'], 'answer': '10 + 100 / (2 * 5)**2 = 11'}) => {'answer': '10 + 100 / (2 * 5)**2 = 11', 'tools_used': ['multiply', 'exponentiate', 'divide', 'add']}


'{"answer": "10 + 100 / (2 * 5)**2 = 11", "tools_used": ["multiply", "exponentiate", "divide", "add"]}'